<a href="https://colab.research.google.com/github/Rizkiramdani04/Portofolio_DS_DA/blob/main/NN_Pytorch_Breast_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using Device: {device}')

Using Device: cpu


In [ ]:
dataset=load_breast_cancer()
X=dataset.data
Y=dataset.target

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(569, 30)
(455, 30)
(114, 30)


In [ ]:
model=StandardScaler()
X_train=model.fit_transform(X_train)
X_test=model.transform(X_test)
type(X_train)

numpy.ndarray

In [ ]:
# convert data to PyTorch tensors and move it to GPU
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
Y_train = torch.tensor(Y_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
Y_test = torch.tensor(Y_test, dtype=torch.float32).to(device)

In [ ]:
class NeuralNet(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(NeuralNet, self).__init__()
    self.fc1=nn.Linear(input_size,hidden_size)
    self.relu=nn.ReLU()
    self.fc2=nn.Linear(hidden_size,output_size)
    self.sigmoid=nn.Sigmoid()
  def forward(self,x):
    out=self.fc1(x)
    out=self.relu(out)
    out=self.fc2(out)
    out=self.sigmoid(out)
    return out

In [ ]:
input_size=X_train.shape[1]
hidden_size=64
output_size=1
learning_rate=0.001
num_epochs=100

In [ ]:
model=NeuralNet(input_size,hidden_size,output_size).to(device)

In [ ]:
criterion=nn.BCELoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()
  outputs=model(X_train)
  loss=criterion(outputs,Y_train.view(-1,1))
  loss.backward()
  optimizer.step()

  with torch.no_grad():
    predicted=outputs.round()
    correct=(predicted==Y_train.view(-1,1)).float().sum()
    accuracy=correct/Y_train.size(0)
  if (epoch+1)%10==0:
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss : {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%")

Epoch [10/100], Loss : 0.5655, Accuracy: 94.29%
Epoch [20/100], Loss : 0.4525, Accuracy: 95.60%
Epoch [30/100], Loss : 0.3626, Accuracy: 95.60%
Epoch [40/100], Loss : 0.2905, Accuracy: 95.82%
Epoch [50/100], Loss : 0.2355, Accuracy: 96.04%
Epoch [60/100], Loss : 0.1953, Accuracy: 96.48%
Epoch [70/100], Loss : 0.1666, Accuracy: 96.48%
Epoch [80/100], Loss : 0.1456, Accuracy: 97.36%
Epoch [90/100], Loss : 0.1299, Accuracy: 97.36%
Epoch [100/100], Loss : 0.1178, Accuracy: 98.02%


In [ ]:
model.eval()
with torch.no_grad():
  outputs=model(X_train)
  predicted=outputs.round()
  correct=(predicted==Y_train.view(-1,1)).float().sum()
  accuracy=correct/Y_train.size(0)
  print(f'Accuracy Traning Data: {accuracy.item()*100:.2f}%')

Accuracy Traning Data: 98.02%


In [ ]:
model.eval()
with torch.no_grad():
  outputs=model(X_test)
  predicted=outputs.round()
  correct=(predicted==Y_test.view(-1,1)).float().sum()
  accuracy=correct/Y_test.size(0)
  print(f'Accuracy Data Test: {accuracy.item()*100:.2f}%')

Accuracy Data Test: 97.37%
